In [3]:
import dis

def g(x):
    return x + 3

dis.dis(g)

  3           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (3)
              4 BINARY_ADD
              6 RETURN_VALUE


CPython's VM is stack-based. It means that it executes instructions using the stack to store and retrieve data.

LOAD_FAST instruction pushes local variable on the stack. 

LOAD_CONST pushes a constant. 

BINARY_ADD pops two objects from the stack, adds them up and pushes the result back.

In [5]:
def g():
    def f(x):
        return x + 1

    print(f(1))

dis.dis(g)

  2           0 LOAD_CONST               1 (<code object f at 0x105158870, file "<ipython-input-5-3acd04a4c5ff>", line 2>)
              2 LOAD_CONST               2 ('g.<locals>.f')
              4 MAKE_FUNCTION            0
              6 STORE_FAST               0 (f)

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                0 (f)
             12 LOAD_CONST               3 (1)
             14 CALL_FUNCTION            1
             16 CALL_FUNCTION            1
             18 POP_TOP
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE

Disassembly of <code object f at 0x105158870, file "<ipython-input-5-3acd04a4c5ff>", line 2>:
  3           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


CPython stores information about what a code block does in a structure called code object. 

To run a module or to call a function means to start evaluating a corresponding code object.

## function object VS code objects

Function objects are created by the execution of the 'def' statement. They reference a code object in their __code__ attribute, which is a purely syntactic object, i.e. nothing more than a compiled version of some source code lines. There is one code object per source code "fragment", but each code object can be referenced by zero or many function objects depending only on how many times the 'def' statement in the source was executed so far.

```
struct PyCodeObject {
    PyObject_HEAD
    int co_argcount;            /* #arguments, except *args */
    int co_posonlyargcount;     /* #positional only arguments */
    int co_kwonlyargcount;      /* #keyword only arguments */
    int co_nlocals;             /* #local variables */
    int co_stacksize;           /* #entries needed for evaluation stack */
    int co_flags;               /* CO_..., see below */
    int co_firstlineno;         /* first source line number */
    PyObject *co_code;          /* instruction opcodes */
    PyObject *co_consts;        /* list (constants used) */
    PyObject *co_names;         /* list of strings (names used) */
    PyObject *co_varnames;      /* tuple of strings (local variable names) */
    PyObject *co_freevars;      /* tuple of strings (free variable names) */
    PyObject *co_cellvars;      /* tuple of strings (cell variable names) */

    Py_ssize_t *co_cell2arg;    /* Maps cell vars which are arguments. */
    PyObject *co_filename;      /* unicode (where it was loaded from) */
    PyObject *co_name;          /* unicode (name, for reference) */
        /* ... more members ... */
};
```

```
typedef struct {
    PyObject_HEAD
    PyObject *func_code;        /* A code object, the __code__ attribute */
    PyObject *func_globals;     /* A dictionary (other mappings won't do) */
    PyObject *func_defaults;    /* NULL or a tuple */
    PyObject *func_kwdefaults;  /* NULL or a dict */
    PyObject *func_closure;     /* NULL or a tuple of cell objects */
    PyObject *func_doc;         /* The __doc__ attribute, can be anything */
    PyObject *func_name;        /* The __name__ attribute, a string object */
    PyObject *func_dict;        /* The __dict__ attribute, a dict or NULL */
    PyObject *func_weakreflist; /* List of weak references */
    PyObject *func_module;      /* The __module__ attribute, can be anything */
    PyObject *func_annotations; /* Annotations, a dict or NULL */
    PyObject *func_qualname;    /* The qualified name */
    vectorcallfunc vectorcall;
} PyFunctionObject;
```

Code object is just a compiled version of source code lines.

Function object is created each time the source is executed. Function objects can linked to same code object(the actual function source code)

## Frame object

```
struct _frame {
    PyObject_VAR_HEAD
    struct _frame *f_back;      /* previous frame, or NULL */
    PyCodeObject *f_code;       /* code segment */
    PyObject *f_builtins;       /* builtin symbol table (PyDictObject) */
    PyObject *f_globals;        /* global symbol table (PyDictObject) */
    PyObject *f_locals;         /* local symbol table (any mapping) */
    PyObject **f_valuestack;    /* points after the last local */

    PyObject **f_stacktop;          /* Next free slot in f_valuestack.  ... */
    PyObject *f_trace;          /* Trace function */
    char f_trace_lines;         /* Emit per-line trace events? */
    char f_trace_opcodes;       /* Emit per-opcode trace events? */

    /* Borrowed reference to a generator, or NULL */
    PyObject *f_gen;

    int f_lasti;                /* Last instruction if called */
    /* ... */
    int f_lineno;               /* Current line number */
    int f_iblock;               /* index in f_blockstack */
    char f_executing;           /* whether the frame is still executing */
    PyTryBlock f_blockstack[CO_MAXBLOCKS]; /* for try and loop blocks */
    PyObject *f_localsplus[1];  /* locals+stack, dynamically sized */
};
```

When a function is called, a new frame is pushed to the VM stack.

Each frame has a reference to the previous frame. (Call stack)



## We've already looked at the three important concepts:

- a code object
- a function object; and
- a frame object.

### CPython has three more:

- a thread state
- an interpreter state; and
- a runtime state.

A thread state is a data structure that contains thread-specific data including the call stack, an exception state and the debugging settings. It should not be confused with an OS thread. They're closely connected

An interpreter state is a group of threads along with the data specific to this group. Threads share such things as loaded modules (sys.modules), builtins (builtins.__dict__) and the import system (importlib).

The runtime state is a global variable. It stores data that is specific to a process. This includes CPython state (is it initialized or not?) and the GIL mechanism.

## Architecture summary
Let's make a quick summary of the CPython's architecture to see how everything fits together. The interpreter can be viewed as a layered structure. The following sums up what the layers are:

- Runtime: global CPython state of a process; includes GIL and memory allocation mechanism
- Interpreter: group of threads and some data they share such as imported modules.
- Thread: data specific to a single OS thread; this includes call stack.
- Frame: element of call stack; provides a state to execute a code object.
- Evaluation loop: executes a code object, which tells what a code block does and contains bytecode and names of variables.

# How CPython Compiler Works

In [11]:
import ast

str = """
x = 123
if x < 3:
  print(666)
"""
tree = ast.parse(str)
print(ast.dump(tree))

Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Constant(value=123, kind=None), type_comment=None), If(test=Compare(left=Name(id='x', ctx=Load()), ops=[Lt()], comparators=[Constant(value=3, kind=None)]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Constant(value=666, kind=None)], keywords=[]))], orelse=[])], type_ignores=[])


```
Module(
body=[Assign(
    targets=[Name(id='x', ctx=Store())], value=Constant(value=123, kind=None), type_comment=None), If(test=Compare(
      left=Name(id='x', ctx=Load()), ops=[Lt()], comparators=[Constant(value=3, kind=None)]
      ), 
      body=[
        Expr(value=Call(func=Name(id='print', ctx=Load()), 
                        args=[Constant(value=666, kind=None)], 
                        keywords=[]))
      ], 
      orelse=[])
  ], 
type_ignores=[])
```